In [1]:
import json
config_path = open("/home/jovyan/work/config.txt")
config_read = config_path.read()
config = json.loads(config_read)

In [2]:
import io
import rasterio
from rasterio.plot import reshape_as_image
import joblib
import numpy as np
import warnings
import matplotlib.pyplot as plt
import numpy as np
import lithops
import pickle
import os
import glob
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.plot import show
from datetime import datetime

# Stackering Lithops | works | get_object, not datastream

The input data is not relevant, in the begining of the function three raster images are open from IBM_COS, the process works successfully and the three bands are stacked.

In [91]:
iterdata  = [('cos://testing-stackering/valenciana_624000.0-4384000.0_comunidad_band3.tif',
             'cos://testing-stackering/valenciana_624000.0-4384000.0_comunidad_band4.tif'),
            ('cos://testing-stackering/valenciana_624000.0-4384000.0_comunidad_band3.tif',
             'cos://testing-stackering/valenciana_624000.0-4384000.0_comunidad_band4.tif')]

In [100]:
def map_stackering(b3, b4, ibm_cos):
    bucket_name='testing-stackering'
    
    raster3 = ibm_cos.get_object(Bucket='testing-stackering', Key='valenciana_624000.0-4384000.0_comunidad_band3.tif')['Body']
    raster4 = ibm_cos.get_object(Bucket='testing-stackering', Key='valenciana_624000.0-4384000.0_comunidad_band4.tif')['Body']
    raster8 = ibm_cos.get_object(Bucket='testing-stackering', Key='valenciana_624000.0-4384000.0_comunidad_band8.tif')['Body']


    with rasterio.open(raster3) as src3:
        with rasterio.open(raster4) as src4:
            with rasterio.open(raster8) as src8:
                
                trueColor = rasterio.open(f'/tmp/raster_stackering.tif','w',driver='Gtiff',
                                 width=src3.width, height=src3.height,
                                 count=3,
                                 crs=src3.crs,
                                 transform=src3.transform,
                                 dtype=src3.dtypes[0]
                                 )
                
                trueColor.write(src3.read(1),3) #blue
                trueColor.write(src4.read(1),2) #green
                trueColor.write(src8.read(1),1) #red
                
                trueColor.close()
        
    #putKey = 'raster_stackering_'+ datetime.now().strftime("%m%d%Y%H%M%S")+'.tif'
    #with open(f'/tmp/raster_stackering.tif', 'rb') as file:
    #    ibm_cos.put_object(Bucket=bucket_name, Key=putKey, Body=file)
    
    with rasterio.open(f'/tmp/raster_stackering.tif') as raster:
        w=raster.read()

    return w

In [101]:
if __name__ == "__main__":
    # For Python 3.7 use mavsonnen/lithops-py37:latest + For Python 3.7 use mavsonnen/jdsampe:nogdal
    fexec = lithops.FunctionExecutor(config=config,runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
    fexec.map(map_stackering, iterdata)
    results = fexec.get_result()

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID 60aa69-21 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID 60aa69-21 | JobID M000 - Uploading function and data - Total: 1.8KiB
ExecutorID 60aa69-21 | JobID M000 - Starting function invocation: map_stackering() - Total: 2 activations
ExecutorID 60aa69-21 - Getting results...


In [102]:
results

[array([[[   0,    0,    0, ..., 2517, 2508, 2491],
         [   0,    0,    0, ..., 2518, 2511, 2512],
         [   0,    0,    0, ..., 2574, 2525, 2496],
         ...,
         [   0,    0,    0, ..., 3170, 3221, 3022],
         [   0,    0,    0, ..., 3106, 3037, 2940],
         [   0,    0,    0, ..., 2965, 3105, 3166]],
 
        [[   0,    0,    0, ..., 1428, 1424, 1442],
         [   0,    0,    0, ..., 1388, 1392, 1411],
         [   0,    0,    0, ..., 1595, 1539, 1359],
         ...,
         [   0,    0,    0, ..., 2133, 2216, 2082],
         [   0,    0,    0, ..., 2164, 2130, 2088],
         [   0,    0,    0, ..., 2069, 2202, 2217]],
 
        [[   0,    0,    0, ...,  863,  831,  865],
         [   0,    0,    0, ...,  859,  827,  844],
         [   0,    0,    0, ...,  954,  897,  793],
         ...,
         [   0,    0,    0, ..., 1496, 1552, 1435],
         [   0,    0,    0, ..., 1443, 1473, 1425],
         [   0,    0,    0, ..., 1432, 1511, 1524]]], dtype=int16),


# Stakering Lithops | NOT work | obj.data_stream

Tratando de leer los archivos de un bucket de ibm_cos me está dando un problemilla la creación del archivo temporal. Si no inicio el archivo temporal con 'w' o 'w+' evidentemente no existe, pero en algun momento tengo que poner 'r+' para que no se cree siempre de nuevo y elimine lo guardado en el mismo. 

Un if con la condición de si existe el archivo temporal, así la primera vez se creará y las siguiente se sobreescribirá. Al tratarse en paralelo me genera confusión que todos las funciones se ejecuten a la vez y todas interpreten que no existe dicho archivo, escribiendo siempre uno nuevo cada uno de ellas.

In [5]:
data_location  = 'cos://testing-stackering'

In [36]:
def stackering_logic(obj, ibm_cos):
    print('I am processing the object cos://{}/{}'.format(obj.bucket, obj.key))

    numberfield=obj.key.split("_")[1]
    numberband=obj.key.split("_")[3].split(".")[0]
    
    
    temporal_file = '/tmp/fa.tif'

    var = os.path.exists(temporal_file)
    
    if obj.key!= 'avalenciana_624000.0-4384000.0_comunidad_band3.tif':
        mode = 'w'
    else:
        mode = 'r+'
            
    with rasterio.open(obj.data_stream) as src:   
        with rasterio.open(temporal_file, mode, driver='Gtiff',
                         width=src.width, height=src.height,
                         count=3,
                         crs=src.crs,
                         transform=src.transform,
                         dtype=src.dtypes[0]
                         ) as trueColor:
            
            if numberband == "band3":
                trueColor.write(src.read(1),3) #blue
            elif numberband == "band4":
                trueColor.write(src.read(1),2) #green
            elif numberband == "band8":
                trueColor.write(src.read(1),1) #red
    
    with rasterio.open(temporal_file) as raster:
        w = raster.read() #w[0]
    
    return [mode,w]

fexec = lithops.FunctionExecutor(config=config, runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
fexec.map(stackering_logic, data_location)
results = fexec.get_result()
results

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID b5d6f1-12 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID b5d6f1-12 | JobID M000 - Uploading function and data - Total: 2.1KiB
ExecutorID b5d6f1-12 | JobID M000 - Starting function invocation: stackering_logic() - Total: 3 activations
ExecutorID b5d6f1-12 - Getting results...


[['w',
  array([[[   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          ...,
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0]],
  
         [[   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          ...,
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0]],
  
         [[   0,    0,    0, ...,  863,  831,  865],
          [   0,    0,    0, ...,  859,  827,  844],
          [   0,    0,    0, ...,  954,  897,  793],
          ...,
          [   0,    0,    0, ..., 1496, 1552, 1435],
          [   0,    0,    0, ..., 1443, 1473, 1425],
          [   0,    0,    0, ..., 1432,

Exception in thread Thread-26:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/lithops/wait/wait_storage.py", line 281, in _future_timeout_checker
    raise TimeoutError('HANDLER', msg)
TimeoutError: [Errno HANDLER] The function did not run as expected.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.8/site-packages/lithops/wait/wait_storage.py", line 290, in _future_timeout_checker
    pickled_exception = str(pickle.dumps(sys.exc_info()))
TypeError: cannot pickle 'traceback' object


## v1

In [8]:
def stackering_logic(obj, ibm_cos):
    print('I am processing the object cos://{}/{}'.format(obj.bucket, obj.key))

    numberfield=obj.key.split("_")[1]
    numberband=obj.key.split("_")[3].split(".")[0]
    
    
    temporal_file = '/tmp/f.tif'

    var = os.path.exists(temporal_file)
    
    if var:
        mode = 'r+'
    else:
        mode = 'w'
            
    with rasterio.open(obj.data_stream) as src:   
        with rasterio.open(temporal_file, mode, driver='Gtiff',
                         width=src.width, height=src.height,
                         count=3,
                         crs=src.crs,
                         transform=src.transform,
                         dtype=src.dtypes[0]
                         ) as trueColor:
            
            if numberband == "band3":
                trueColor.write(src.read(1),3) #blue
            elif numberband == "band4":
                trueColor.write(src.read(1),2) #green
            elif numberband == "band8":
                trueColor.write(src.read(1),1) #red
    
    with rasterio.open(temporal_file) as raster:
        w = raster.read() #w[0]
    
    return [mode,w]

fexec = lithops.FunctionExecutor(config=config, runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
fexec.map(stackering_logic, data_location)
results = fexec.get_result()
results

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID b5d6f1-3 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID b5d6f1-3 | JobID M000 - Uploading function and data - Total: 2.0KiB
ExecutorID b5d6f1-3 | JobID M000 - Starting function invocation: stackering_logic() - Total: 3 activations
ExecutorID b5d6f1-3 - Getting results...


[['r+',
  array([[[   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          ...,
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0],
          [   0,    0,    0, ...,    0,    0,    0]],
  
         [[   0,    0,    0, ..., 1428, 1424, 1442],
          [   0,    0,    0, ..., 1388, 1392, 1411],
          [   0,    0,    0, ..., 1595, 1539, 1359],
          ...,
          [   0,    0,    0, ..., 2133, 2216, 2082],
          [   0,    0,    0, ..., 2164, 2130, 2088],
          [   0,    0,    0, ..., 2069, 2202, 2217]],
  
         [[   0,    0,    0, ...,  863,  831,  865],
          [   0,    0,    0, ...,  859,  827,  844],
          [   0,    0,    0, ...,  954,  897,  793],
          ...,
          [   0,    0,    0, ..., 1496, 1552, 1435],
          [   0,    0,    0, ..., 1443, 1473, 1425],
          [   0,    0,    0, ..., 1432

## v2

Creating the file outside the map function , it is said: no such file or directory. If its inside it reconogice the file.

In [31]:
temporal_file = '/tmp/def_file_stack2.tif'
temporal_file = '/home/jovyan/work/Training_stuff/Exp_txt/ha.tif'


with rasterio.open('/home/jovyan/work/Training_stuff/Exp_stack/valenciana_624000.0-4384000.0_comunidad_band3.tif') as src:
    meta=src.meta
    meta.update(count = 3)
    
with rasterio.open(temporal_file, 'w+', **meta) as data:
    print('creado')

creado


In [32]:
def stackering_logic(obj, ibm_cos):
    print('I am processing the object cos://{}/{}'.format(obj.bucket, obj.key))

    numberfield=obj.key.split("_")[1]
    numberband=obj.key.split("_")[3].split(".")[0]
            
    with rasterio.open(obj.data_stream) as src:   
        with rasterio.open(temporal_file, 'r+') as trueColor:
            
            if numberband == "band3":
                trueColor.write(src.read(1),3) #blue
            elif numberband == "band4":
                trueColor.write(src.read(1),2) #green
            elif numberband == "band8":
                trueColor.write(src.read(1),1) #red
    
    with rasterio.open(temporal_file) as raster:
        w = raster.read() #w[0]
    
    return w

fexec = lithops.FunctionExecutor(config=config, runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
fexec.map(stackering_logic, data_location)
results = fexec.get_result()
results

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID b5d6f1-9 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID b5d6f1-9 | JobID M000 - Uploading function and data - Total: 1.8KiB
ExecutorID b5d6f1-9 | JobID M000 - Starting function invocation: stackering_logic() - Total: 3 activations
ExecutorID b5d6f1-9 - Getting results...


TypeError: /home/jovyan/work/Training_stuff/Exp_txt/ha.tif: No such file or directory

# Stackering Local | works |

Version de lectura de todos los archivos en local, funciona correctamente, misma implementación.

In [269]:
input_path = '/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Experiment_lithops/'
criteria = 'valenciana*.tif'
all_files = glob.glob(input_path + criteria)
all_files

['/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Experiment_lithops/valenciana_624000.0-4384000.0_comunidad_band3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Experiment_lithops/valenciana_624000.0-4384000.0_comunidad_band4.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Experiment_lithops/valenciana_624000.0-4384000.0_comunidad_band8.tif']

In [270]:
a='/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Experiment_lithops/valenciana_624000.0-4384000.0_comunidad_band3.tif'
numberfield=a.split("/")[7].split("_")[1]
numberfield

'624000.0-4384000.0'

In [287]:
for raster_path in all_files:

    numberfield=raster_path.split("/")[7].split("_")[1]
    numberband=raster_path.split("/")[7].split("_")[3].split(".")[0]
    
    #temporal_file='/home/jovyan/work/Training_stuff/output_stack/stack'+ numberfield +'.tif'    
    temporal_file='/tmp/raster_temporalnew22222.tif'
    
    
    if os.path.isfile(temporal_file):
        mode = 'r+'
    else:
        mode = 'w' 
    
    with rasterio.open(raster_path) as src:
        
        with rasterio.open(temporal_file,mode,driver='Gtiff',
                             width=src.width, height=src.height,
                             count=3,
                             crs=src.crs,
                             transform=src.transform,
                             dtype=src.dtypes[0]
                             ) as trueColor:
            
            if numberband == "band3":
                trueColor.write(src.read(1),3) #blue
            elif numberband == "band4":
                trueColor.write(src.read(1),2) #green
            elif numberband == "band8":
                trueColor.write(src.read(1),1) #red

In [288]:
#temporal_file='/home/jovyan/work/Training_stuff/output_stack/stack624000.0-4384000.0.tif' 
a=rasterio.open(temporal_file)
print(a.read())
np.unique(a.read())

[[[   0    0    0 ... 2517 2508 2491]
  [   0    0    0 ... 2518 2511 2512]
  [   0    0    0 ... 2574 2525 2496]
  ...
  [   0    0    0 ... 3170 3221 3022]
  [   0    0    0 ... 3106 3037 2940]
  [   0    0    0 ... 2965 3105 3166]]

 [[   0    0    0 ... 1428 1424 1442]
  [   0    0    0 ... 1388 1392 1411]
  [   0    0    0 ... 1595 1539 1359]
  ...
  [   0    0    0 ... 2133 2216 2082]
  [   0    0    0 ... 2164 2130 2088]
  [   0    0    0 ... 2069 2202 2217]]

 [[   0    0    0 ...  863  831  865]
  [   0    0    0 ...  859  827  844]
  [   0    0    0 ...  954  897  793]
  ...
  [   0    0    0 ... 1496 1552 1435]
  [   0    0    0 ... 1443 1473 1425]
  [   0    0    0 ... 1432 1511 1524]]]


array([  -89,   -87,   -86, ..., 10232, 11002, 11479], dtype=int16)

In [163]:
os.path.isfile(temporal_file) 

True

# similar with .txt file

In [106]:
path = '/home/jovyan/work/Training_stuff/Exp_txt/aacuatro.txt'
f = open(path,'r')
z=f.read()

['ahora tenemos cuatro palabras',
 'en esta frase tenemos seis palabras',
 'si me pongo a escribir tanto voy a llegar a las trece palabras',
 'una palabra',
 'si quieres más serán cinco']

In [140]:
def map_model(obj):
    print('Bucket: {}'.format(obj.bucket))
    print('Key: {}'.format(obj.key))
    print('Partition num: {}'.format(obj.part))

    
    data = obj.data_stream.read()
                
    a=data.splitlines() 
    
        # Writing results to file
    with open('/tmp/file3.txt', 'w') as file:
        file.write(str(a))
        
        
    with open('/tmp/file3.txt', 'r') as file:
        info = file.read()
        
        
    

    return info

In [141]:
iterdata = ['cos://testing-us-east/primero.txt',
            'cos://testing-us-east/segundo.txt',
            'cos://testing-us-east/tercero.txt']

fexec = lithops.FunctionExecutor(config=config,runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
fexec.map(map_model, iterdata)
results = fexec.get_result()
results

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID b5d6f1-67 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID b5d6f1-67 | JobID M000 - Uploading function and data - Total: 1.4KiB
ExecutorID b5d6f1-67 | JobID M000 - Starting function invocation: map_model() - Total: 3 activations
ExecutorID b5d6f1-67 - Getting results...


["[b'esta oracion primera']",
 "[b'aqui la segunda']",
 "[b'en la tercera ', b'hacemos un salto']"]

# other stuff

In [20]:
filename = '/tmp/raster.tif'
os.path.isfile(filename) 
 
if os.path.isfile(filename) :
    print('exists')
else:
    print('does not exists')

does not exists


In [22]:
data_location  = 'cos://testing-stackering'

def my_map_function(obj):
    print('Bucket: {}'.format(obj.bucket))
    print('Key: {}'.format(obj.key))
    print('Partition num: {}'.format(obj.part))
    counter = 0
    
    filename = '/tmp/hola.txt'
    val = os.path.exists(filename) 
    
    if obj.key == 'valenciana_624000.0-4384000.0_comunidad_band4.tif':
        with open('/tmp/hola.txt', 'w') as file:
            file.write('new doc')

    return(val)

fexec = lithops.FunctionExecutor(config=config, runtime='mavsonnen/jdsampe:nogdal', runtime_memory=2048)
fexec.map(my_map_function, data_location)
results = fexec.get_result()
results

Lithops v2.2.5 init for IBM Cloud Functions - Namespace: new-us-east - Region: us_east
ExecutorID b5d6f1-5 | JobID M000 - Selected Runtime: mavsonnen/jdsampe:nogdal - 2048MB 
ExecutorID b5d6f1-5 | JobID M000 - Uploading function and data - Total: 1.6KiB
ExecutorID b5d6f1-5 | JobID M000 - Starting function invocation: my_map_function() - Total: 3 activations
ExecutorID b5d6f1-5 - Getting results...


[False, True, False]

In [291]:
road_to_file = '/home/jovyan/work/Training_stuff/Exp_stack/STACK624000.0-4384000.0 (1).tif'
raster_on_road = rasterio.open(road_to_file)
raster_on_road.read()


array([[[   0,    0,    0, ..., 2517, 2508, 2491],
        [   0,    0,    0, ..., 2518, 2511, 2512],
        [   0,    0,    0, ..., 2574, 2525, 2496],
        ...,
        [   0,    0,    0, ..., 3170, 3221, 3022],
        [   0,    0,    0, ..., 3106, 3037, 2940],
        [   0,    0,    0, ..., 2965, 3105, 3166]],

       [[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]]], dtype=int16)